https://stepik.org/lesson/573090/step/2

In [ ]:
# 0.48

# Решение

In [ ]:
x = [0, 1, 2]
y = [1, 2, 3]
h = 0.1
C = 1
a = [0, 0]
epohs = 2

y_pred = lambda x: a[1] * x + a[0]

# Эпохи с подсчётом новых w_0, w_1:
for epoh in range(epohs):
    grad = [0, 0]
    # Подсчёт суммы для L_grad:
    for i in range(len(x)):
        grad[0] += 2 * (y_pred(x[i]) - y[i])
        grad[1] += 2 * (y_pred(x[i]) - y[i]) * x[i]
    # Регуляризация:
    grad[0] += 2 * C * a[0]
    grad[1] += 2 * C * a[1]
    # Новые w_0, w_1
    a[0] -= h * grad[0]
    a[1] -= h * grad[1]
    print(f'epoh = {epoh+1}, w_0 = {round(a[0], 2)}, w_1 = {round(a[1], 2)}')


epoh = 1, w_0 = 1.2, w_1 = 1.6
epoh = 2, w_0 = 0.48, w_1 = 0.56


# Форум

In [ ]:
#@title Матрицы numpy:
# Vadim Kopeykin https://stepik.org/lesson/573090/step/2?discussion=8583570&thread=solutions&unit=567639
import numpy as np

X = np.array([[1, 0], [1, 1], [1, 2]])
y = np.array([1, 2, 3])

w = np.array([0., 0.])
C = 1
h = 0.1
for i in range(2):
    grad_w = 2 * (X @ w - y) @ X + 2 * C * w
    w -= grad_w * h
print(w[0])

In [ ]:
#@title Тензорная арифметика с использованием PyTorch:
# Артём Ионаш https://stepik.org/lesson/573090/step/2?discussion=8471756&thread=solutions&unit=567639

import torch

x = torch.tensor([[0], [1], [2]], dtype=torch.float32)
y_true = torch.tensor([[1], [2], [3]], dtype=torch.float32)

w0 = torch.tensor([0], requires_grad=True, dtype=torch.float32)
w1 = torch.tensor([0], requires_grad=True, dtype=torch.float32)

h = 0.1  # learning_rate
с = 1.0  # 0.5 * weight_decay
num_epochs = 2

print(f"a{0}: ({w0.item()},  {w1.item()})")
for i in range(num_epochs):
    # Вычисление члена L2-регуляризации и функции потерь L
    regularization_term = с * (torch.sum(w0**2) + torch.sum(w1**2))
    L = torch.sum((x * w1 + w0 - y_true) ** 2) + regularization_term

    # Вычисление градиентов и обновление параметров с помощью градиентного спуска
    grad_w0, grad_w1 = torch.autograd.grad(L, [w0, w1])
    with torch.no_grad():
        w0 -= h * grad_w0
        w1 -= h * grad_w1
        print(f"a{i}: ({w0.item()},  {w1.item()})")

In [ ]:
#@title pandas + sympy для генерации L
# Igor Kovbel https://stepik.org/lesson/573090/step/2?discussion=7515878&unit=567639

import pandas as pd
import sympy as sp

df = pd.DataFrame({'X': [0, 1, 2], 'Y': [1, 2, 3]})
w1, w0 = sp.symbols('w1, w0')

a = [0, 0]
h = 0.1
C = 1

L = ''
for i, (X, y) in enumerate(zip(df['X'], df['Y'])):
    L += f'({X}*{w1}+{w0}-{y})**2'
    if i+1 != df.shape[0]:
        L += '+'
L += f'+{C}*({w0}**2+{w1}**2)'

diff_w1 = sp.diff(L, w1)
diff_w0 = sp.diff(L, w0)

for i in range(2):
    a = [a[0] - h * diff_w0.evalf(subs={w0: a[0], w1: a[1]}),
         a[1] - h * diff_w1.evalf(subs={w0: a[0], w1: a[1]})]
    print(f'Iteration {i+1}: {a}')


Iteration 1: [1.20000000000000, 1.60000000000000]
Iteration 2: [0.480000000000000, 0.560000000000000]


In [ ]:
# Artem Kutsan https://stepik.org/lesson/573090/step/2?discussion=8487382&thread=solutions&unit=567639
from sympy import symbols, diff

# Задаем входные данные: нецелевой признак (X) и целевой признак (Y)
X = [0, 1, 2]
Y = (1, 2, 3)

# Создаем символьные переменные
w0, w1 = symbols('w0 w1')

# Задаем константу регуляризации
C = 1

# Формула для среднеквадратичной ошибки (MSE) с регуляризацией
L = sum((x*w1 + w0 - y)**2 for x, y in zip(X, Y)) + C*(w0**2 + w1**2)

# a - начальная точка для оптимизации, steps - количество шагов и h - размер шага градиентного спуска
a, steps, h = [(0, 0)], 2, 0.1

# Градиентный спуск: расчитываем значения w0 и w1 на каждом шаге
[a.append([a[n][k] - h * diff(L, v).subs({w0: a[n][0], w1: a[n][1]}).evalf() for k, v in enumerate(['w0', 'w1'], 0)]) for n in range(steps)]

# Вывод результата
print(*[f'a{i}: {tuple(map(lambda a: round(float(a), 2), a[i]))}' for i in range(len(a))], sep='\n')
print(f'Значение первой координаты (которая соотвтетствует весу w0) позиции a2: {round(float(a[2][0]), 2)}')

a0: (0.0, 0.0)
a1: (1.2, 1.6)
a2: (0.48, 0.56)
Значение первой координаты (которая соотвтетствует весу w0) позиции a2: 0.48


In [ ]:
#@title sp.utilities.lambdify
# Alex Vasilchenko https://stepik.org/lesson/573090/step/2?discussion=5128625&unit=567639
import sympy as sp
w0, w1 = sp.symbols('w0 w1')
L = (w0-1)**2 + (w1 + w0 - 2)**2 + (2*w1 + w0 -3)**2 + w0**2 + w1**2

dLdw1 = sp.utilities.lambdify(
    (w0, w1),
    sp.diff(L, w1),
)
dLdw0 = sp.utilities.lambdify(
    (w0, w1),
    sp.diff(L, w0),
)

h = 0.1
a_w0 = 0
a_w1 = 0
n = 2

for i in range(0, n + 1):
    print(f'a{i}: ({a_w0},  {a_w1})')
    ai_w0 = a_w0 - h * dLdw0(a_w0, a_w1)
    ai_w1 = a_w1 - h * dLdw1(a_w0, a_w1)
    a_w0 = ai_w0
    a_w1 = ai_w1

a0: (0,  0)
a1: (1.2000000000000002,  1.6)
a2: (0.47999999999999987,  0.5599999999999994)
